In [1]:
from pymongo import MongoClient
import pprint
from textblob import TextBlob, __version__
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
__version__
from googletrans import Translator

In [26]:
client = MongoClient('mongodb://localhost:27017/myDB')
db = client['mydb']
wall_posts = db['wall_posts']

analyzer = SentimentIntensityAnalyzer()
s=""
for post in wall_posts.find({}):
    vs = analyzer.polarity_scores(post['text'])
    if vs['pos']> 0.0:
        print("\n\n", "********************************************************************************" )
        print(post['text'])
        print(vs)
        print( "********************************************************************************" )
   
    



 ********************************************************************************
Это цитата сообщения LUADMILA Оригинальное сообщениеВсё, что у меня  было          LUADMILA                                       ФAll that I had is gone   And the sad songs know how,   I feel Your kind of love was never enough   To brighten the melodies   All that I have is gone   Packed my bags and just disappeared   And I search for the street   Where we used to meet   To find you again my dear.    Все, что у меня было, пропало   И печальные песни знают,   Как  я  себя  чувствую.  Твоей любви никогда не хватало,   Чтобы  скрасить  мелодии   Все,  что  у  меня  есть,  ушло.   Упаковал  мои  сумки  и  просто  исчез.   И  я  ищу  улицу,  где  мы  встречались,   Чтобы  снова  найти  тебя,  мой дорогой.    Kovacs - Shirley (Sound Of The Underground)               https://www.youtube.com/watch?time_continue=67&amp;v=UErFMNfOEbQ
{'neg': 0.064, 'neu': 0.88, 'pos': 0.056, 'compound': 0.0261}
*****************

In [87]:


import time


pos_count = 0
pos_correct = 0

with open("C:\\Users\\kirill\\Desktop\\project_case\\positive.txt","r") as f:
    for line in f.read().split('\n'):
        vs = analyzer.polarity_scores(line)
        if not vs['neg'] > 0.1:
            if vs['pos']-vs['neg'] > 0:
                pos_correct += 1
            pos_count +=1


neg_count = 0
neg_correct = 0

with open("C:\\Users\\kirill\\Desktop\\project_case\\negative.txt","r") as f:
    for line in f.read().split('\n'):
        vs = analyzer.polarity_scores(line)
        if not vs['pos'] > 0.1:
            if vs['pos']-vs['neg'] <= 0:
                neg_correct += 1
            neg_count +=1

print("Positive accuracy = {}% via {} samples".format(pos_correct/pos_count*100.0, pos_count))
print("Negative accuracy = {}% via {} samples".format(neg_correct/neg_count*100.0, neg_count))

Positive accuracy = 80.69370058658507% via 3921 samples
Negative accuracy = 91.73643975245722% via 2747 samples


In [25]:
from googletrans import Translator
trr = Translator()
trr.detect('')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)